In [4]:
import folium
from folium import FeatureGroup, LayerControl
import json
import math
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point, MultiPolygon
import geopandas as gpd



In [5]:
cities = pd.read_csv("data/cities.csv", sep=" ", names=["lon", "lat", "city"])
temperatures = pd.read_csv("data/temperature.txt", sep=" ", names=["lon", "temp", "days", "day"])
troops = pd.read_csv("data/troops.txt", sep=" ", names=["lon", "lat", "survivors", "direction", "division"])

In [6]:
df_troops = pd.DataFrame(troops)
geometry = [Point(xy) for xy in zip(df_troops.lon, df_troops.lat)]
crs = {'init': 'epsg:4326'}
gdf_troops = gpd.GeoDataFrame(troops, crs=crs, geometry=geometry)

In [7]:
coords = []
for i, row in gdf_troops.iterrows():
    coords.append([row.geometry.y, row.geometry.x])

min_lon, min_lat = [coord * .98 for coord in min(coords)] 
max_lon, max_lat = [coord * 1.02 for coord in max(coords)] 

center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

In [8]:
m = folium.Map(location=[54,31],
               tiles='OpenStreetMap',
               zoom_start=6,
               control_scale=True)




In [9]:
def draw_polygons(mapobj, gdf_troops):
    
    for i, row in gdf_troops.iterrows():
        
        if i == len(gdf_troops) - 1:
            pass
        
        else:
            
            
            x1, y1 = coords[i]
            x2, y2 = coords[i+1]

            if x2 - x1 != 0:
                #slope of line
                slope = (y2 - y1) / (x2 - x1)
                #negative inverse slope (perpindicular angle)
                slope_inv = (-1 / slope)
            else:
                slope_inv = 0

            #cosine, sine to find x, y coordinates of polygon corners
            c = 1 / math.sqrt(1 + (slope_inv**2))
            s = slope_inv / math.sqrt(1 + (slope_inv**2))

            #find width of polygon (magnitude of survivors)
            gdf_survivors = gdf_troops['survivors']
            #abstract max survivors
            #move outside of function and make new column
            buffer = (gdf_survivors[i] / 34000) *.03

            x1u, y1u = (x1 + (buffer*c)), (y1 + (buffer*s))
            x1d, y1d = (x1 - (buffer*c)), (y1 - (buffer*s))
            x2u, y2u = (x2 + (buffer*c)), (y2 + (buffer*s))
            x2d, y2d = (x2 - (buffer*c)), (y2 - (buffer*s))

            lat_point_list = [(x1 + (buffer*c)), (x1 - (buffer*c)), (x2 - (buffer*c)), (x2 + (buffer*c))]
            lon_point_list = [(y1 + (buffer*s)), (y1 - (buffer*s)), (y2 - (buffer*s)), (y2 + (buffer*s))]     
            
            fg_attack = folium.FeatureGroup(name='attack').add_to(mapobj)
            fg_retreat = folium.FeatureGroup(name='retreat').add_to(mapobj)

            geometry = list(zip(lat_point_list, lon_point_list))


            if gdf_troops['direction'][i] == 'A':
                
                folium.Polygon(geometry,
                           color = '#F0B683',
                           fill_color = '#F0B683', 
                           fill_opacity = 1).add_to(fg_attack)    
                
                
            else:
                 
                folium.Polygon(geometry,
                           color = 'black',
                           fill_color = 'black', 
                           fill_opacity = 1).add_to(fg_retreat)
            
            mapobj.keep_in_front(fg_attack)
    
    return mapobj
    
    

In [10]:
draw_polygons(m, gdf_troops)